In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import load_model
import pickle

In [4]:
## Load the trained model , scaler pickle , onehot....

model = load_model('model.h5')

## load the encoder and scaler
with open('ohe_geography.pkl' , 'rb') as file:
    ohe_geography = pickle.load(file)

with open('label_encoder_gender.pkl' , 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl' , 'rb') as file:
    scaler = pickle.load(file)


In [5]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [ ]:
# ## One-Hot Encode 'Geography'...

geo_encoded = ohe_geography.transform([[input_data['Geography']]])  # ✅ no .toarray()
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=ohe_geography.get_feature_names_out(['Geography'])
)
geo_encoded_df

d:\All ML PROJECT\ANN End to End Project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_Germany,Geography_Spain
0,0.0,0.0


In [10]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [11]:
#3 ENcode the Categorical Variables...
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [12]:
## Concatenation Gender and Geography column
input_df = pd.concat([input_df.drop("Geography" , axis=1) , geo_encoded_df] , axis =1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,0.0,0.0


In [14]:
## Scaled the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221, -0.57946723,
        -0.57638802]])

In [15]:
## Predict the Churn
Prediction = model.predict(input_scaled)
Prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step


array([[0.03795121]], dtype=float32)

In [18]:
prediction_proba = Prediction[0][0]
prediction_proba

np.float32(0.037951212)

In [19]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
